In [189]:
import pandas as pd
import pymysql
from datetime import datetime


def act_log_data():
    conn = pymysql.connect(host='116.213.69.213',port=3306,user='taomin',
                           passwd='KnHJZu-6Bm',db='action',charset='utf8')
    cur = conn.cursor(pymysql.cursors.SSCursor)
    cur.execute('''select * from action_log where time BETWEEN '1458921600' and '1458921604'
                ''')#1459094399
    act_data = [[row[8], row[10], row[2], row[3]] for row in cur]
    df = pd.DataFrame(act_data, columns=['uuid', 'uid', 'url', 'date'])
    return df


def pv(df):
    grouped = df.groupby(['date', 'url'])
    pv_d = {}
    for k, v in grouped:
        pv_d.setdefault(k[0],{})[k[1]] = len(v['uuid'].tolist())
            
    pv = [[datetime.fromtimestamp(date).strftime('%Y-%m-%d %H'), 
               pv_d[date][k]] for date in pv_d 
                                  for k in pv_d[date]]
    df = pd.DataFrame(pv, columns = ['date', 'pv'])
    return df


def uv(df):
    grouped = df.groupby(['date', 'url'])
    uv_d = {}
    for k, v in grouped:
        if v['uid'].tolist() == ['']:
            continue
        uv_d.setdefault(k[0],{})[k[1]] = len(v['uid'].tolist())
            
    uv = [[datetime.fromtimestamp(date).strftime('%Y-%m-%d %H'), 
               uv_d[date][k]] for date in uv_d 
                                  for k in uv_d[date]]
    df = pd.DataFrame(uv, columns = ['date', 'uv'])
    return df


def time_pv(df_pv):
    grouped = df_pv.groupby('date')
    time_pv = [[k, v['pv'].sum()] for k, v in grouped]
    return time_pv


def time_uv(df_uv):
    grouped = df_uv.groupby('date')
    time_uv = [[k, v['uv'].sum()] for k, v in grouped]
    return time_uv


#time_pv(pv(act_log_data()))
time_uv(uv(act_log_data()))

[['2016-03-26 00', 17]]